In [1]:
import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime, date
from collections import defaultdict, OrderedDict

## Read the CSV, parse the 24/7 and late fields into a single field


In [2]:
restaurants = pandas.read_csv('restaurant_database.csv')

restaurants['open'] = ['24/7' if value == 'yes' else 'early' for value in restaurants['24/7?']]
restaurants['open'] = ['late' if (current == 'early' and open_late == 'yes') else current
                       for current, open_late 
                       in zip(restaurants['open'], restaurants['Open late (after 10pm)'])]

## Define translation functions

We take the knowledge base, as defined in the CSV (downloaded from Google sheets), and define how to translate it to a Prolog KB. This includes what should be the names for conditions which are boolean (not multivalued), various measured of input sanitization, and other renamings.

In [55]:
def sanitze_to_atom(name, prefix='w'):
    sanitized = name.lower().replace(' ', '_').replace('-', '_').replace('?', '').replace(',', '')
    if sanitized[0].isdigit():
        sanitized = prefix + sanitized        
    return sanitized


BOOLEAN_AFFIRMATIVE_NAME = {
    'coffee': 'serves',
    'alcohol': 'serves',
    'vegeterian': 'friendly',
    'pescaterian': 'friendly',
    'take_away': 'offers'
}


COLUMNS_TO_IGNORE = (
    'Name', 'Address', 'Open late (after 10pm)', '24/7?'
)


TRUE_VALUES = ('yes', 'y', 'True', 'true', True)


class BooleanTransformer:
    def __init__(self, col_atom):
        self.col_atom = col_atom
        
    def __call__(self, value):
        out_value = BOOLEAN_AFFIRMATIVE_NAME[self.col_atom]
        if not value in TRUE_VALUES:
            out_value = 'not_' + out_value
        
        return '{name}({value})'.format(name=self.col_atom, value=out_value)

    
class MultivaluedTransformer:
    def __init__(self, col_atom):
        self.col_atom = col_atom
        
    def __call__(self, value):
        return '{name}({value})'.format(name=self.col_atom, 
                                        value=sanitze_to_atom(value))

    
def build_transformers(knowledge_base):
    transformers = {}
    multivalued = []
    
    for column in knowledge_base.columns:
        if column in COLUMNS_TO_IGNORE:
            continue
        
        column_atom = sanitze_to_atom(column)
        if column_atom in BOOLEAN_AFFIRMATIVE_NAME:
            transformers[column] = BooleanTransformer(column_atom)
            
        else:
            transformers[column] = MultivaluedTransformer(column_atom)
            multivalued.append(column)
            
    return transformers, multivalued


INFORMATIVE_FIELDS = ('Name', 'Address', 'Neighborhood')
            
    
def transform_kb(knowledge_base):
    transformers, multivalued = build_transformers(knowledge_base)
    restaurants = {}
    output = []
    
    for rest_id, rest_row in knowledge_base.iterrows():
        transformed_values = [transformers[col](rest_row[col]) for col in transformers]
        output.append('suggest(rest_{id}) :- {cond}.'.format(id=rest_id, 
                                                             cond=', '.join(transformed_values)))
        
        restaurants['rest_{id}'.format(id=rest_id)] = {field: rest_row[field] 
                                                       for field in INFORMATIVE_FIELDS}
    
    output.append('')
    for col in transformers:
        col_atom = sanitze_to_atom(col)
        output.append('{col}(X) :- check({col}, X).'.format(col=col_atom))
    
    output.append('')
    output.extend(['multivalued({col}).'.format(col=sanitze_to_atom(col)) for col in multivalued])
    
    return output, restaurants
    
        

In [56]:
output, rests = transform_kb(restaurants)
print('\n'.join(output))

suggest(rest_0) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(japanese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(friendly), open(early), price_range(w7500_15000).
suggest(rest_1) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(korean), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(friendly), open(early), price_range(w5000_7500).
suggest(rest_2) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(taiwanese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(friendly), open(early), price_range(w15000_25000).
suggest(rest_3) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(korean), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(not_friendly), open(late), price_range(w5000_7500).
suggest(rest_4) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(korean), alcohol(serves), take_away(not_offers), coffee(not_serves),

In [57]:
KB = """
% Enter your KB below this line:

suggest(rest_0) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(japanese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(friendly), open(early), price_range(w7500_15000).
suggest(rest_1) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(korean), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(friendly), open(early), price_range(w5000_7500).
suggest(rest_2) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(taiwanese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(friendly), open(early), price_range(w15000_25000).
suggest(rest_3) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(korean), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(not_friendly), open(late), price_range(w5000_7500).
suggest(rest_4) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(korean), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_friendly), open(late), price_range(w7500_15000).
suggest(rest_5) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(italian), alcohol(serves), take_away(not_offers), coffee(serves), pescaterian(friendly), open(late), price_range(w15000_25000).
suggest(rest_6) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(korean), alcohol(not_serves), take_away(not_offers), coffee(serves), pescaterian(not_friendly), open(early), price_range(w7500_15000).
suggest(rest_7) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(breakfast), alcohol(serves), take_away(not_offers), coffee(serves), pescaterian(friendly), open(late), price_range(w7500_15000).
suggest(rest_8) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(vietnamese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_friendly), open(w24/7), price_range(w7500_15000).
suggest(rest_9) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(vietnamese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_friendly), open(late), price_range(w7500_15000).
suggest(rest_10) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(korean), alcohol(not_serves), take_away(not_offers), coffee(not_serves), pescaterian(not_friendly), open(early), price_range(w7500_15000).
suggest(rest_11) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(japanese), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(not_friendly), open(w24/7), price_range(w5000_7500).
suggest(rest_12) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(vietnamese), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(not_friendly), open(early), price_range(w5000_7500).
suggest(rest_13) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(japanese), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_friendly), open(late), price_range(w25000+).
suggest(rest_14) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(american), alcohol(serves), take_away(offers), coffee(serves), pescaterian(not_friendly), open(early), price_range(w7500_15000).
suggest(rest_15) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(japanese), alcohol(serves), take_away(offers), coffee(not_serves), pescaterian(friendly), open(early), price_range(w7500_15000).
suggest(rest_16) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(japanese), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(friendly), open(early), price_range(w7500_15000).
suggest(rest_17) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(american), alcohol(not_serves), take_away(offers), coffee(serves), pescaterian(friendly), open(w24/7), price_range(w7500_15000).
suggest(rest_18) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(american), alcohol(not_serves), take_away(offers), coffee(serves), pescaterian(friendly), open(w24/7), price_range(w7500_15000).
suggest(rest_19) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(korean), alcohol(serves), take_away(offers), coffee(not_serves), pescaterian(not_friendly), open(early), price_range(w7500_15000).
suggest(rest_20) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(chinese), alcohol(serves), take_away(offers), coffee(not_serves), pescaterian(friendly), open(early), price_range(w7500_15000).
suggest(rest_21) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(korean), alcohol(serves), take_away(offers), coffee(not_serves), pescaterian(friendly), open(late), price_range(w7500_15000).
suggest(rest_22) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(korean), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_friendly), open(w24/7), price_range(w7500_15000).
suggest(rest_23) :- neighborhood(itaewon), vegeterian(not_friendly), cuisine_type(turkish), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(not_friendly), open(early), price_range(w5000_7500).
suggest(rest_24) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(american), alcohol(not_serves), take_away(offers), coffee(serves), pescaterian(friendly), open(late), price_range(w15000_25000).
suggest(rest_25) :- neighborhood(gangnam), vegeterian(not_friendly), cuisine_type(korean), alcohol(serves), take_away(not_offers), coffee(not_serves), pescaterian(not_friendly), open(early), price_range(w7500_15000).
suggest(rest_26) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(korean), alcohol(serves), take_away(offers), coffee(not_serves), pescaterian(friendly), open(w24/7), price_range(w5000_7500).
suggest(rest_27) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(japanese), alcohol(not_serves), take_away(offers), coffee(not_serves), pescaterian(friendly), open(early), price_range(w7500_15000).

neighborhood(X) :- check(neighborhood, X).
vegeterian(X) :- check(vegeterian, X).
cuisine_type(X) :- check(cuisine_type, X).
alcohol(X) :- check(alcohol, X).
take_away(X) :- check(take_away, X).
coffee(X) :- check(coffee, X).
pescaterian(X) :- check(pescaterian, X).
open(X) :- check(open, X).
price_range(X) :- check(price_range, X).

multivalued(neighborhood).
multivalued(cuisine_type).
multivalued(price_range).
multivalued(open).
neighborhood(X) :- check(neighborhood, X).
vegeterian(X) :- check(vegeterian, X).
cuisine_type(X) :- check(cuisine_type, X).
alcohol(X) :- check(alcohol, X).
take_away(X) :- check(take_away, X).
coffee(X) :- check(coffee, X).
pescaterian(X) :- check(pescaterian, X).
open(X) :- check(open, X).
price_range(X) :- check(price_range, X).

multivalued(neighborhood).
multivalued(cuisine_type).
multivalued(price_range).
multivalued(open).

% Care-checking implemented to allow the user to ignore attributes they don't care about:

check(A, X) :- not(check_if_care(A, X)).
check_if_care(A, X) :- care(A), not(ask(A, X)).
care(A) :- atom_concat(care_check_, A, X), ask(X, '').

% Asking clauses

ask(A, V):-
known(y, A, V), % succeed if true
!. % stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

ask(A, V):-
not(multivalued(A)),
% write_py(A:not_multivalued),
known(y, A, V2),
V \== V2,
!, fail.

ask(A, V):-
read_py(A,V,Y), % get the answer
asserta(known(Y, A, V)), % remember it
Y == y. % succeed or fail
"""

with open("KB_A.pl", "w") as text_file:
    text_file.write(KB)

In [59]:
# The code here will ask the user for input based on the askables
# It will check if the answer is known first

from pyswip.prolog import Prolog
from pyswip.easy import *

prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True


# This dictionary maps a variable name to a string template 
# of how to ask the user about it in a friendly manner
DEFAULT_QUESTION = '{A} is {V}? '
ATOM_TO_QUESTION_MAPPING = defaultdict(lambda: DEFAULT_QUESTION)
ATOM_TO_QUESTION_MAPPING.update({
    'neighborhood': 'Do you wish to eat in the {V} {A} ? ',
    'vegeterian': 'Should the restaurant be {A} {V}? ',
    'pescaterian': 'Should the restaurant be {A} {V}? ',
    'cuisine_type': 'Do you wish to eat {V} food tonight? ',
    'alcohol': 'Do you need a place that {V} {A}? ',
    'coffee': 'Do you need a place that {V} {A}? ',
    'take_away': 'Do you need a place that {V} take-away? ',
    'open': 'Would you like a place that is {A} {V}? ',
    'price_range': 'Are you okay with a price range of {V} per person? '
})

NEGATION_PREFIX = 'not_'
NEGATION_REPLACEMENT = 'does not '

CARE_CHECK_PREFIX = 'care_check_'
DEFAULT_CARE_QUESTION = 'Do you care about {A}? '
CARE_CHECK_QUESTION_MAPPING = defaultdict(lambda: DEFAULT_CARE_QUESTION)
CARE_CHECK_QUESTION_MAPPING.update({
    'neighborhood': 'Do you care which {A} you eat in? ',
    'vegeterian': 'Do you care if the restaurant is {A} friendly? ',
    'pescaterian': 'Do you care if the restaurant is {A} friendly? ',
    'cuisine_type': 'Do you care about which cuisine you eat tonight? ',
    'alcohol': 'Do you care if the restaurant serves {A}? ',
    'coffee': 'Do you care if the restaurant serves {A}? ',
    'take_away': 'Do you care if the restaurant offers take-away? ',
    'open': 'Do you care if the restaurant is open late or 24/7? ',
    'price_range': 'Do you care about expected price range of the restaurant? '
})

YES_VALUES = ('yes', 'y', 'true', 'sure', 'okay', 'definitely', 'cool')

def read_py(A,V,Y):
    A_value = A.get_value() if type(A) == Atom else str(A)
    V_value = V.get_value() if type(V) == Atom else str(V)
    
    if A_value.startswith(CARE_CHECK_PREFIX):
        A_value = A_value[len(CARE_CHECK_PREFIX):]
        question = CARE_CHECK_QUESTION_MAPPING[A_value]
   
    else:
        question = ATOM_TO_QUESTION_MAPPING[A_value]
        if V_value.startswith(NEGATION_PREFIX):
            V_value = NEGATION_REPLACEMENT + V_value[len(NEGATION_PREFIX):-1]
    
    if A_value == 'price_range':
        V_value = '₩' + V_value[1:].replace('_', '-')
        
    user_input = raw_input(question.format(A=A_value, V=V_value))
    
    if user_input.lower() in YES_VALUES:
        user_input = 'y'
    
    Y.unify(user_input)
    return True


write_py.arity = 1
read_py.arity = 3

registerForeign(read_py)
registerForeign(write_py)

prolog.consult("KB_A.pl") # open the KB
call(retractall(known))
found = False

for soln in prolog.query("suggest(X).", maxresult=1):
    rest = soln['X']
    info = rests[rest]
    print('Your restaurant reccomendation is {name} at {address} in {neighborhood}'.format(
        name=info['Name'], address=info['Address'], neighborhood=info['Neighborhood']))
    
    found = True
    
if not found:
    print('Sorry, no acceptable restaurants found, please try again.')

Do you care which neighborhood you eat in? n
Do you care if the restaurant is vegeterian friendly? n
Do you care about which cuisine you eat tonight? y
Do you wish to eat japanese food tonight? n
Do you wish to eat korean food tonight? n
Do you wish to eat taiwanese food tonight? n
Do you wish to eat italian food tonight? n
Do you wish to eat breakfast food tonight? n
Do you wish to eat vietnamese food tonight? y
Do you care if the restaurant serves alcohol? n
Do you care if the restaurant offers take-away? y
Do you need a place that does not offer take-away? y
Do you care if the restaurant serves coffee? y
Do you need a place that does not serve coffee? y
Do you care if the restaurant is pescaterian friendly? n
Do you care if the restaurant is open late or 24/7? n
Do you care about expected price range of the restaurant? n
Your restaurant reccomendation is EMOI Pho at Gangnam station, Exit 3 in Gangnam
